####  Notebook 3B
This notebook fits a randomforest model on training images

In [1]:
# organize imports
from __future__ import print_function


import numpy as np
import h5py
import os
import json
import pickle

import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.externals import joblib
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV

In [2]:
# load the user configs
with open('room_recognition/conf/conf_randomforest.json') as f:    
  config = json.load(f)

# config variables

seed      = config["seed"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
results     = config["results"]
classifier_path = config["classifier_path"]
num_classes   = config["num_classes"]
model_path=config["model_path"]

In [3]:
# import features and labels
h5f_data  = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

trainfeatures = np.array(features_string)
trainlabels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()

In [4]:
# verify the shape of features and labels
print ("[INFO] features shape: {}".format(trainfeatures.shape))
print ("[INFO] labels shape: {}".format(trainlabels.shape))



[INFO] features shape: (1040, 2048)
[INFO] labels shape: (1040,)


In [5]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [6]:
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
pprint(param)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [50, 266, 483, 700, 916, 1133, 1350, 1566, 1783, 2000]}


instantiate the random search and fit it

In [8]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

In [9]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param, n_iter = 100, cv = 3, verbose=2, random_state=seed, n_jobs = -1)

In [10]:
# Fit the random search model
rf_random.fit(trainfeatures, trainlabels)



Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV] n_estimators=1133, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False 
[CV]  n_estimators=1133, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False, total=  43.7s
[CV] n_estimators=1133, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True, total=  45.6s
[CV] n_estimators=1133, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstra

[CV] n_estimators=1783, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False 
[CV]  n_estimators=916, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=False, total= 1.8min
[CV] n_estimators=1566, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=True 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.7min


[CV]  n_estimators=1566, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=True, total= 1.9min
[CV] n_estimators=1566, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=True 
[CV]  n_estimators=1783, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False, total= 3.5min
[CV] n_estimators=1566, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=True 
[CV]  n_estimators=1783, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False, total= 3.5min
[CV] n_estimators=266, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=110, bootstrap=True 
[CV]  n_estimators=266, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=110, bootstrap=True, total=  12.6s
[CV] n_estimators=266, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=110, bootstrap=True 
[CV]  n_estimators=1783,

[CV] n_estimators=1566, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1783, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total= 3.7min
[CV] n_estimators=1566, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1783, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total= 3.5min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=60, bootstrap=False 
[CV]  n_estimators=1566, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False, total= 3.0min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=60, bootstrap=False 
[CV]  n_estimators=1566, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False, total= 3.0min
[CV] n_estimator

...

[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=50, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=40, bootstrap=False, total=   1.8s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=True, total=  47.0s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=True, total=  46.6s
[CV] n_estimators=916, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=False 
[CV]  n_estimators=916, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=False, total=  32.3s
[CV] n_estimators=916,

[CV] n_estimators=916, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=1350, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, total= 1.6min
[CV] n_estimators=916, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=1350, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, total= 1.6min
[CV] n_estimators=1566, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=40, bootstrap=False 
[CV]  n_estimators=1350, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, total= 1.6min
[CV] n_estimators=1566, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=40, bootstrap=False 
[CV]  n_estimators=916, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=True, total=  43.6s
[CV] n_estimators=1566, 

[CV] n_estimators=483, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False 
[CV]  n_estimators=483, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False, total=  35.5s
[CV] n_estimators=1783, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=True 
[CV]  n_estimators=700, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=60, bootstrap=False, total=  50.8s
[CV] n_estimators=1783, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=True 
[CV]  n_estimators=700, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=60, bootstrap=False, total=  49.4s
[CV] n_estimators=1783, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=True 
[CV]  n_estimators=483, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False, total=  34.7s
[CV] n_estimators=50, min

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 76.8min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [50, 266, 483, 700, 916, 1133, 1350, 1566, 1783, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=9, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [11]:
rf_random.best_params_


{'n_estimators': 1350,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': False}

In [12]:
rf_random.best_score_


0.8519230769230769

In [13]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [ 75, 80],
    'max_features': ['sqrt' ],
    'min_samples_leaf': [4, 6],
    'min_samples_split': [ 5, 7 ],
    'n_estimators': [ 1200, 1350]
}


In [14]:
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
clf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
    

In [15]:
clf.fit(trainfeatures, trainlabels)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200 
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200 
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200 
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200, total= 1.3min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200, total= 1.3min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_esti

[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 16.7min


[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350, total= 1.2min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1200 
[CV]  bootstrap=Fals

[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1200, total= 1.1min
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1200, total=  58.1s
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1200, total=  58.0s
[CV] bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1350 
[CV]  bootstrap=False, max_depth=75, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1350, total= 1.1min
[CV] bootstrap=False

[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200, total= 1.0min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200, total= 1.1min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200, total= 1.1min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200, total= 1.0min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=1200 
[CV]  bootstrap=Fals

[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350, total= 1.1min
[CV] bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, n_estimators=1200 
[CV]  bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1350, total= 1.1min
[CV] bootstrap=False

[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 50.5min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [False], 'max_depth': [75, 80], 'max_features': ['sqrt'], 'min_samples_leaf': [4, 6], 'min_samples_split': [5, 7], 'n_estimators': [1200, 1350]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [16]:
clf.best_params_

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 1350}

In [17]:
clf.best_score_

0.8509615384615384

In [19]:
 joblib.dump(clf.best_estimator_, model_path)

['room_recognition/output/randomforest/model']